# Hybrid Mode - Dataset Creation 
we want to use the following structure to train our model:

Input:
- idea_text: "AI-powered e-commerce platform..."
- static_features: [market_size, investment, competition, team_strength]
- historical_stock: [month_1, month_2, ..., month_36]

Target:
- stock_performance (next 12 months): [month_37, month_38, ..., month_48]

---


This should then produce the follwoing output with our model:

Input:
- idea_text: "AI-powered e-commerce platform..."

Optional Input:
- static_features: [market_size, investment, competition, team_strength]

Output:
- Predicted stock performance (next 12 months): [month_1, month_2, ..., month_12]


### Huggingface Dataset
We will use the shortbread/tickers dataset to get tickers of US companies

In [3]:
from datasets import load_dataset

# Load the dataset
def get_tickers():
    initial_dataset = load_dataset("shortbread/tickers", split="train")

    # Select only the required columns and rename 'long_business_summary' to 'idea'
    updated_dataset = initial_dataset.remove_columns([col for col in initial_dataset.column_names if col not in ['symbol']])
    
    dataset_array = updated_dataset['symbol']

    # Display a few rows of the filtered and renamed dataset
    print(dataset_array)
    return dataset_array 

### Fetching additional data
We use the dataset from Hugginface as startingpoint and will now provide methods to fetch additional data

In [4]:
import pandas as pd
import yfinance as yf
import os
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import joblib

# Function to fetch stock performance data
def fetch_stock_performance(ticker, months=12):
    try:
        # Fetch historical stock data with 'max' period
        stock = yf.Ticker(ticker)
        hist = stock.history(period="max", interval="1mo")  # Get all available monthly data

        # Check if history is empty
        if hist.empty:
            print(f"$ {ticker}: no price data found (possibly delisted or insufficient history).")
            return None

        # Adjust the range to the available data if it's less than the requested months
        if len(hist) < months:
            print(f"$ {ticker}: insufficient data for {months} months, only {len(hist)} months available.")
            months = len(hist)

        # Extract the last `months` of closing prices
        monthly_close = hist['Close'][-months:].tolist()
        return monthly_close

    except Exception as stock_e:
        print(f"Failed to get ticker '{ticker}' reason: {stock_e}")
        return None


# Function to fetch business description
def fetch_business_description(ticker):
    try:
        stock = yf.Ticker(ticker)
        description = stock.info.get("longBusinessSummary", None)
        return description
    except Exception as description_e:
        print(f"Error fetching business description for {ticker}: {description_e}")
        return None

# Function to fetch market size (revenue)
def fetch_market_size(ticker):
    try:
        stock = yf.Ticker(ticker)
        # Revenue (Total Revenue TTM)
        revenue = stock.financials.loc["Total Revenue"].iloc[0]  # Most recent revenue
        return revenue
    except Exception as market_size_e:
        print(f"Error fetching market size for {ticker}: {market_size_e}")
        return None

# Function to fetch investment (total assets)
def fetch_investment(ticker):
    try:
        stock = yf.Ticker(ticker)
        # Total assets (most recent)
        total_assets = stock.balance_sheet.loc["Total Assets"].iloc[0]
        return total_assets
    except Exception as investment_e:
        print(f"Error fetching investment for {ticker}: {investment_e}")
        return None

# Function to fetch team strength (number of employees)
def fetch_team_strength(ticker):
    try:
        stock = yf.Ticker(ticker)
        # Number of full-time employees
        employees = stock.info.get("fullTimeEmployees", 0)
        return employees
    except Exception as team_e:
        print(f"Error fetching team strength for {ticker}: {team_e}")
        return None

# Main function to create the dataset
def create_real_dataset(tickers, months):
    data = []
    for ticker in tickers:
        print(f"Processing {ticker}...")

        # Fetch stock performance
        stock_performance = fetch_stock_performance(ticker, months)

        # Handle cases where stock performance has fewer months than required
        if stock_performance is not None and len(stock_performance) < months:
            # Pad with None from the front to match the required length
            stock_performance = [None] * (months - len(stock_performance)) + stock_performance

        # Fetch business description
        description = fetch_business_description(ticker)

        # Extract static features
        market_size = fetch_market_size(ticker)
        investment = fetch_investment(ticker)
        team_strength = fetch_team_strength(ticker)

        if None in [description, market_size, investment, team_strength]:
            print(f'We skip ${ticker}, because one of the values is Null.')
            continue

        # Combine all data into a dictionary
        data.append({
            "ticker": ticker,
            "business_description": description,
            "market_size": market_size,
            "investment": investment,
            "team_strength": team_strength,
            **{f"month_{i+1}_performance": sp for i, sp in enumerate(stock_performance or [None] * months)}
        })

    return pd.DataFrame(data)


def process_tickers_in_batches(tickers, batch_size=50, output_file="Data/real_company_stock_dataset.csv", months=24):
    # Process tickers in batches
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        print(f"Processing batch {i // batch_size + 1} of {len(tickers) // batch_size + 1}...")

        # Process the batch using the existing method
        batch_data = create_real_dataset(batch, months=months)

        # Save the batch to the output file
        if not batch_data.empty:
            if not os.path.exists(output_file):
                print("Creating output file...")
                batch_data.to_csv(output_file, index=False)
            else:
                print("Appending to output file...")
                batch_data.to_csv(output_file, mode='a', header=False, index=False)

        print(f"Batch {i // batch_size + 1} processed and saved.")
        
        # Add a delay between batches to respect API rate limits
        time.sleep(10)  # Adjust as needed

def normalize_full_dataset(input_file="Data/real_company_stock_dataset.csv",
                           output_file="Data/normalized_real_company_stock_dataset.csv",
                           add_to_existing=True):
    dataset = pd.read_csv(input_file)

    # Identify static and historical columns
    static_columns = ['market_size', 'investment', 'team_strength']
    historical_columns = [col for col in dataset.columns if col.startswith('month_')]

    # Paths for scalers
    static_scaler_path = "Data/Scalar/static_scaler.pkl"
    historical_scaler_path = "Data/Scalar/historical_scaler.pkl"

    if add_to_existing and os.path.exists(static_scaler_path) and os.path.exists(historical_scaler_path):
        print("ADD_TO_EXISTING is True. Loading existing scalers...")
        # Load existing scalers
        static_scaler = joblib.load(static_scaler_path)
        historical_scaler = joblib.load(historical_scaler_path)

        # Transform the dataset using the existing scalers
        dataset[static_columns] = static_scaler.transform(dataset[static_columns])
        dataset[historical_columns] = historical_scaler.transform(dataset[historical_columns])

    else:
        print("Starting fresh. Creating new scalers...")
        # Initialize new scalers
        static_scaler = StandardScaler()  # or MinMaxScaler()
        historical_scaler = MinMaxScaler(feature_range=(0, 1))

        # Fit scalers on the entire dataset
        dataset[static_columns] = static_scaler.fit_transform(dataset[static_columns])
        dataset[historical_columns] = historical_scaler.fit_transform(dataset[historical_columns])

    # Save the normalized dataset
    dataset.to_csv(output_file, index=False)

    # Save the scalers for future use
    joblib.dump(static_scaler, static_scaler_path)
    joblib.dump(historical_scaler, historical_scaler_path)

    print("Normalization complete. Scalers updated.")



ADD_TO_EXISTING = True
OUTPUT_FILE = "Data/real_company_stock_dataset.csv"
NORMALIZED_OUTPUT_FILE = "Data/normalized_real_company_stock_dataset.csv"

# Example usage
if __name__ == "__main__":
    # Simulated list of 10,000 tickers
    fetched_tickers = get_tickers()
    
    if ADD_TO_EXISTING and os.path.exists(OUTPUT_FILE):
        try:
            existing_tickers = set(pd.read_csv(OUTPUT_FILE, usecols=[0])['ticker'])
            fetched_tickers = [ticker for ticker in fetched_tickers if ticker not in existing_tickers]
            print("Filtered Tickers")
            
        except Exception as e:
            print(f"Error loading existing tickers: {e}")
            
    # Process tickers in batches
    process_tickers_in_batches(fetched_tickers, output_file=OUTPUT_FILE, batch_size=50, months=24)
    
    # Normalize data
    normalize_full_dataset(input_file=OUTPUT_FILE, output_file=NORMALIZED_OUTPUT_FILE, add_to_existing=ADD_TO_EXISTING)
    print("Dataset creation complete!")


['A', 'AA', 'AAC', 'AACG', 'AACI', 'AACT', 'AADI', 'AAIC', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'AAU', 'AB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABEV', 'ABG', 'ABIO', 'ABL', 'ABLV', 'ABM', 'ABNB', 'ABOS', 'ABR', 'ABSI', 'ABST', 'ABT', 'ABUS', 'ABVC', 'AC', 'ACA', 'ACAB', 'ACAC', 'ACAD', 'ACAH', 'ACAQ', 'ACAX', 'ACB', 'ACBA', 'ACCD', 'ACCO', 'ACDC', 'ACEL', 'ACER', 'ACET', 'ACGL', 'ACGLO', 'ACGN', 'ACHC', 'ACHL', 'ACHR', 'ACHV', 'ACI', 'ACIC', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACNT', 'ACON', 'ACOR', 'ACP', 'ACR', 'ACRE', 'ACRO', 'ACRS', 'ACRV', 'ACRX', 'ACST', 'ACT', 'ACTG', 'ACU', 'ACV', 'ACVA', 'ACWV', 'ACXP', 'ADAG', 'ADAP', 'ADBE', 'ADC', 'ADCT', 'ADD', 'ADEA', 'ADER', 'ADES', 'ADEX', 'ADI', 'ADIL', 'ADM', 'ADMA', 'ADMP', 'ADN', 'ADNT', 'ADOC', 'ADP', 'ADPT', 'ADRT', 'ADSE', 'ADSK', 'ADT', 'ADTH', 'ADTN', 'ADTX', 'ADUS', 'ADV', 'ADVM', 'ADX', 'ADXN', 'AE', 'AEAE', 'AEE', 'AEF', 'AEG', 'AEHL', 'AEHR', 'AEI', 'AEI

$AAC: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


Processing AACG...
Processing AACI...


$AACI: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
AACT: Period '5y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', 'ytd', 'max']


Processing AACT...
Processing AADI...
Processing AAIC...


...

Processing ZYNE...


Failed to get ticker 'ZYNE' reason: Expecting value: line 1 column 1 (char 0)
$ZYNE: possibly delisted; no price data found  (period=5y)


Processing ZYXI...


$ZYXI: possibly delisted; no price data found  (period=5y)


Batch 74 processed and saved.
Normalization complete and scalers saved.
Dataset creation complete!
